In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q peft~=0.14.0 trl~=0.13.0

# 訓練問答模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練問答模型。我們將使用 `transformers` 套件中的 `SFTTrainer` ([Supervised Fine-tuning Trainer](https://huggingface.co/docs/trl/sft_trainer)) 類別來訓練模型，。

In [ ]:
# import garbage collector
import gc

import pandas as pd

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
)
from datasets import load_dataset
from transformers import (
  pipeline,
)

from typing import Any
from pydantic import BaseModel
from pprint import pprint

import torch

# 載入 PEFT 相關套件
from peft import LoraConfig, TaskType, PeftModel, get_peft_model
# 載入 SFTTrainer 相關套件
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM, setup_chat_format

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

In [3]:
# The full `train` split, only 1% of dataset
immutable_dataset = load_dataset("philschmid/dolly-15k-oai-style", split="train[:1%]")

### 資料包含什麼？

In [ ]:
# 顯示原始資料中包含的 features 以及筆數
immutable_dataset

In [ ]:
# 檢視資料集中的第一筆資料
pprint(immutable_dataset[0]['messages'])

In [ ]:
# 將 messages 欄位分拆成 user 和 assistant 兩個欄位，方便演示
dataset = immutable_dataset.map(
  lambda x: {
    "user": x["messages"][0],
    "assistant": x["messages"][1],
  }
)
# 顯示處理後的資料
dataset

In [ ]:
# 顯示前 first_n_data 筆資料
first_n_data = 3
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset.select(range(first_n_data)))

## 訓練設定


In [8]:
# 訓練相關設定, 利用降低 batch size 提高 gradient accumulation steps 來節省記憶體
class Config(BaseModel):
  model_name: str = 'facebook/opt-2.7b'
  torch_dtype: Any = torch.bfloat16 # 半精度浮點數
  adam_epsilon: float = 1e-4 # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  saved_model_path: str = 'sample_data/saved_encoder_model' # path to save the trained model
  saved_lora_path: str = 'sample_data/saved_lora_model' # path to save the trained LORA model
  train_batch_size: int = 2 # size of the input batch in training
  gradient_accumulation_steps: int = 2 # number of updates steps to accumulate before performing a backward/update pass
  epochs: int = 20 # number of times to iterate over the entire training dataset
  lr: float = 1e-3 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.

  # Pipieline 相關設定
  pipeline_name: str = 'text-generation' # pipeline name
  temperature: float = 0.1 # temperature for sampling
  max_new_tokens: int = 125 # 限制最大生成字數
  repetition_penalty: float = 1.5 # 重複機率, 1~2 之間, 1.0 (no penalty), 2.0 (maximum penalty)

  # LORA 相關設定
  rank: int = 128 # rank of the LORA model
  lora_alpha: int = 32 # alpha of the LORA model

if device.type == 'mps': # 方便在 Apple Silicon 上快速測試
  config = Config(
    model_name='facebook/opt-125m',
    torch_dtype=torch.float32, # 在 Apple Silicon 若使用預訓練模型 opt-125m 需要使用全精度浮點數，否則會出現錯誤
    epochs=3,
  )
else:
  config = Config()

## 先觀察 Fine-tuning 前的表現

### 載入 Tokenizer

In [ ]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
# 檢視 Tokenizer，是否存在 padding token 及 padding side 等資訊
pprint(tokenizer)

### 載入預訓練模型

In [10]:
model = AutoModelForCausalLM.from_pretrained(
  config.model_name,
  torch_dtype=config.torch_dtype,
  low_cpu_mem_usage=True,
).to(device)

In [ ]:
pprint(model)

### 詠唱格式化 (Prompt Formatting)

先定義我們的詠唱 (Prompt) 格式。為此，我們將創建一個格式化函數。

In [ ]:
# Set up the chat format with default 'chatml' format
if tokenizer.chat_template is None:
  model, tokenizer = setup_chat_format(model, tokenizer)
  print('=== 設定 chat format ===')
  pprint(tokenizer)

In [13]:
def instruction_formatter(x):
  return tokenizer.apply_chat_template(
    [x['user']],
    tokenize=False,
    add_generation_prompt=True,
  )

In [14]:
input = instruction_formatter(dataset[0])

In [ ]:
pprint(input)

### Fine-tuning 前的表現

In [16]:
# 載入預訓練模型
generator = pipeline(
  task=config.pipeline_name,
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 顯示預訓練模型預測結果
response = generator(
  input,
  temperature=config.temperature,
  max_new_tokens=config.max_new_tokens,
  repetition_penalty=config.repetition_penalty,
)
pprint(response[0]['generated_text'])

## 訓練模型

隨著 `trl` 的最新版本發布，現在支持流行的指令 (instruction) 和對話 (conversation) 數據集格式。這意味著我們只需要將數據集轉換為支持的格式之一，`trl` 會處理其餘的部分。這些格式包括：

* 指令格式 instruction format

```json
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

* 對話格式 conversational format

```json
{"messages": [{"role": "system", "content": "You are..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
```

我們所準備的資料集恰巧符合對話格式，因此我們可以直接使用它。

In [ ]:
# 顯示單筆方便閱讀
pprint(dataset[0]['messages'])

### LoRA 的訓練策略 - 降維打擊

LoRA（Low-Rank Adaptation）是一種用於訓練大型語言模型的技術，旨在提高訓練效率並減少計算資源的需求。以下是為何需要透過LoRA訓練的一些原因：

降低計算成本：LoRA 通過將模型的權重矩陣分解為低秩矩陣，顯著減少了參數的數量，從而降低了計算成本和內存需求。

加速訓練速度：由於參數數量減少，LoRA 可以加速模型的訓練過程，使得在相同的硬件資源下能夠更快地完成訓練。

![](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png)

In [ ]:
# 查看預訓練模型可訓練的參數量，其數量相當龐大，所以需要透過 Low Rank Approximation (LORA) 來降低參數量
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

#### PEFT 配置

`lora_alpha`: 決定小模型的影響程度，也就是說 Alpha 值越高，越容易把大模型既有的能力給覆蓋掉。

`target_module`: 要降維的模型層，可以透過 `model.named_parameters` 查看。

對於許多語言微調任務，由於引入了新標記 (Token)，擴展模型的詞彙表是必要的。這需要擴展嵌入層 (Embedding) 以考慮新標記 (Token)，並在保存適配器時存儲嵌入層 (Embedding) 以及適配器權重。

通過將嵌入層 (Embedding) 添加到配置的 `target_modules` 中來保存嵌入層 (Embedding)。嵌入層 (Embedding) 的名稱必須遵循 Transformers 的標準命名方案。

In [ ]:
# PEFT 配置
lora_config = LoraConfig(
  task_type=TaskType.CAUSAL_LM,
  r=config.rank,
  lora_alpha=config.lora_alpha,
  # target_modules=[
  #   'embed_tokens', 'lm_head', # embedding layer
  #   'q_proj', 'k_proj', 'v_proj', 'out_proj', # attention layer
  # ],
)
pprint(lora_config)

#### 取得 PEFT 模型

搭配預訓模型及 PEFT 配置，我們可以取得 PEFT 模型。我們可以觀察受到降維影響的模型層。

In [ ]:
# 取得 PEFT 模型
peft_model = get_peft_model(
  model, # 預訓練模型
  lora_config, # PEFT 配置
)

In [ ]:
pprint(lora_config)

In [ ]:
# 取得 PEFT 模型, 觀察受 PEFT 影響的模型參數
peft_model

#### 調整 PEFT 模型精度

PEFT 模型的精度是 `torch.float32`，我們可以透過 `model.half()` 將其轉換為半精度。

In [ ]:
# 獲取 PERF 模型參數名稱及型態，確認是否使用半精度浮點數
for name, param in peft_model.named_parameters():
  print(f'{name}: {param.dtype}')

In [25]:
if config.torch_dtype == torch.float16 or config.torch_dtype == torch.bfloat16:
  peft_model = peft_model.half() # 轉換為半精度浮點數

In [ ]:
# 查看可訓練的參數量
peft_model.print_trainable_parameters()

### 定義訓練參數

In [27]:
def formatting_prompts_func(x):
  return tokenizer.apply_chat_template(
    x['messages'],
    tokenize=False,
    add_generation_prompt=False,
  )

In [28]:
input = formatting_prompts_func(dataset[0])

In [ ]:
pprint(input)

In [30]:
response_template = '<|im_start|>assistant'
collator = DataCollatorForCompletionOnlyLM(
  tokenizer=tokenizer,
  response_template=response_template,
)

In [ ]:
training_args = SFTConfig(
  output_dir='sample_data/train_output_pii_masking',
  learning_rate=config.lr,
  per_device_train_batch_size=config.train_batch_size,
  gradient_accumulation_steps=config.gradient_accumulation_steps,
  num_train_epochs=config.epochs,
  weight_decay=config.weight_decay,
  save_strategy='epoch', # 每個 epoch 儲存一次
  report_to='none', # Disable wandb on colab
  adam_epsilon=config.adam_epsilon, # 當使用半精度浮點數時，需要設定較大的 adam epsilon
  packing=False, # Disable packing when using DataCollatorForCompletionOnlyLM
)

trainer = SFTTrainer(
    model=peft_model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
)


### 開始訓練

In [ ]:
# 開始訓練，這可能需要一些時間
trainer.train()

#### 保存 LoRA 模型參數

In [33]:
# # 保存 Lora 参数
# peft_model.save_pretrained(
#   config.saved_lora_path,
#   # warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
#   save_embedding_layers=True,
# )

#### 合併 LoRA 模型參數

In [34]:
# # 合併原始模型和 Lora 参数
# new_model = PeftModel.from_pretrained(model, config.saved_lora_path)

# # print("=== 合併前的模型結構 ===")
# print(new_model)

In [35]:
# # 合併並卸載 Lora 参数
# new_model.merge_and_unload()

# # print("=== 合併後的模型結構 ===")
# print(new_model)

In [36]:
# # 保存合併後的模型
# new_model.save_pretrained(config.saved_model_path)
# tokenizer.save_pretrained(config.saved_model_path)

### 釋放資源

In [37]:
# # 釋放 GPU 記憶體
# del new_model
# del trainer

# peft_model.to('cpu')
# del peft_model

# model.to('cpu')
# del model
# torch.cuda.empty_cache()

# gc.collect()

## 評估模型

### 載入微調後 Tokenizer

從已經完成訓練的模型取得 Tokenizer，可以留意這個訓練時保存下來的 Tokenizer 仍保有訓練時的設定，包涵 `pad_token` 和 `padding_side`。

In [38]:
# tokenizer = AutoTokenizer.from_pretrained(
#   config.saved_model_path
# )
# # 檢視 Tokenizer
# pprint(tokenizer)

### 載入微調後模型

以半精度浮點數載入已經完成訓練的模型

In [ ]:
# 以半精度浮點數載入已經完成訓練的模型
# model = AutoModelForCausalLM.from_pretrained(
#   config.saved_model_path,
#   low_cpu_mem_usage=True,
#   torch_dtype=config.torch_dtype,
# ).to(device)

### Fine-tuning 後的表現

In [40]:
# 載入新模型
generator = pipeline(
  task=config.pipeline_name,
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [41]:
pipeline.model = peft_model

In [ ]:
# 顯示新模型預測結果
input = instruction_formatter(dataset[0])
response = generator(
  input,
  temperature=config.temperature,
  max_new_tokens=config.max_new_tokens,
  repetition_penalty=config.repetition_penalty,
)
pprint(response[0]['generated_text'])